In [37]:
from random import randint, uniform
from util import generate_problem, verify_witness

In [119]:
def random_gene(T, r):
    num_variables = max([max(t['i'], t['j']) for t in T])
    return [0] + [randint(-r, r) for _ in range(num_variables)]

def sweep_line(intervals):
    L = []
    for l, r in intervals:
        L.append( (l, 1) )
        L.append( (r, -1) )
        
    L.sort(key=lambda a: 10*a[0] - a[1])
    
    c = 0
    top = -1
    x = 0
    
    for p, i in L:
        c += i
        if c > top:
            top = c
            x = p
    
    # returns the number of constraints unsatisfied and time point
    return len(intervals) - top, x

def walk_gene(gene, T):
    num_variables = max([max(t['i'], t['j']) for t in T])
    
    constraints_per_variable = [list() for i in range(num_variables + 1)]
    # at index i should contain availability intervals (given other variables are fixed.)
    # remark: existence of an interval is local with respect to one constraint
    # remark: so if intersection is empty, then there is no "safe" place to put this variable
    
    for c in T:
        i, j = c['i'], c['j']
        intervals = c['intervals']
        for l, r in intervals:
            # X_j - X_i in (l, r)
            # fix X_j to genes[j]
            # so genes[j] - X_i in (l, r)
            # then X_i in [X_j - r, X_j - l]
            constraints_per_variable[i].append( (gene[j] - r, gene[j] - l) )
            
            # now for j:
            constraints_per_variable[j].append( (gene[i] + l, gene[i] + r) )
            
            # ^ tested this on paper, it's ok.
    
    
    # for each constraint, sweep line, find best option
    # (later? we want to pick whatever REDUCES the unsatisfieds the most)
    # for now: pick a random variable and modify it according to sweep_line
    # if no later, then we can make the loop upstairs simpler (to only do stuff if "i" is mentioned)
    i = randint(1, num_variables)
    gene[i] = sweep_line(constraints_per_variable[i])[1]

    return gene
    
def walk_tcsp(T, r, max_iterations, max_flips):
    best_gene = None
    best_gene_failed = None
    
    for i in range(max_iterations):
        gene = random_gene(T, r)
        
        for j in range(max_flips):
            gene_failed = verify_witness(gene, T)
            print(len(gene_failed))
            
            if not best_gene or len(gene_failed) < len(best_gene_failed):
                best_gene = gene
                best_gene_failed = gene_failed
            
            if len(gene_failed) == 0:
                break
                
            gene = walk_gene(gene, T)
                
        # necessary for double break after flips loop
        if len(gene_failed) == 0: break
    
    print(f'num iterations: {i+1}')
    print('best gene:', best_gene)
    print('constraints failed:', len(best_gene_failed), 'out of:', len(T)) 
    print('failed constraints:', best_gene_failed)

In [122]:
from pprint import pprint
T = generate_problem(20, scaling_factor=1)
walk_tcsp(T, 10, 50, 50)

25
25
22
22
22
22
19
17
17
17
15
13
12
12
12
12
11
10
8
8
8
5
5
5
5
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
23
21
21
19
17
17
17
16
16
16
15
15
13
13
12
11
11
11
11
11
10
10
10
10
10
10
10
10
10
10
10
10
10
8
8
8
8
8
8
8
8
7
7
7
7
7
7
7
7
7
26
21
19
18
16
14
14
14
10
10
10
10
10
9
9
8
8
8
7
7
7
7
6
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
6
5
5
5
5
5
5
5
5
5
5
24
19
17
17
15
15
15
14
11
11
11
10
9
8
8
7
7
6
6
6
6
6
7
7
7
7
7
6
5
5
5
5
5
5
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
25
21
20
17
16
14
12
11
10
10
10
9
9
9
9
9
9
9
9
9
9
8
8
8
8
8
8
8
8
8
8
6
7
7
7
7
7
7
7
7
7
7
7
7
6
6
6
6
6
6
28
27
27
25
23
21
21
20
20
17
14
13
13
10
9
9
9
6
6
6
6
6
6
6
6
5
5
4
4
4
4
4
4
4
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
24
23
21
18
17
16
15
15
14
14
13
11
11
10
9
9
9
9
9
9
9
8
8
8
8
8
8
8
8
8
8
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
22
21
18
18
18
17
16
14
14
14
14
14
13
12
10
10
10
9
9
9
9
9
8
8
8
8
8
8
8
8
7
7
6
6
6
6
6
5
5
5
5
5
5
5
5
5
5
5
5
5
26
25
21
21
17
15
15
14
14
14
14
11
10
8
8
7
7
7
7
6
6
6
6
6
6
6
6
6